In [1]:
#In this file we create tensorboard log directory to logs or track the error file in it!!!
#and also creating model_checkpoint_directory to save the best model file after training done

In [2]:
from pathlib import Path
import os,sys
from src.ImageClassifier.Exception import CustomException
from src.ImageClassifier.loggers import logger
from src.ImageClassifier.Constants import *
from src.ImageClassifier.Utils import Save_Model,Create_Directory,read_yaml_file
from dataclasses import dataclass
import tensorflow

## step3) update the entity files 

In [3]:
#in these entity file we r defining the parameter which is used in yaml file as class variable
@dataclass
class CallBackModelConfig():
    # define the parameters
    root_dir_path:Path
    tensorboard_log_dir_path:Path
    model_checkpoint_path:Path

## step4) upate the configurationManager file which is present in src/config/configuration.py

In [4]:
#In the configurationmanager we file we read the yaml file and create directories
# and also initializing the value to 
#class variable CallBackModelConfig class andtaking rtn as function
class ConfigurationManager():
    def __init__(self,config_filepath=CONFIG_FILEPATH,param_filepath=PARAM_FILEPATH):
        #reading the yaml file it rtn value as configbox dictatonay
        self.config = read_yaml_file(config_filepath)
        self.param = read_yaml_file(param_filepath) #rtn value as configbox dictaonary

        #create artifacts directory
        Create_Directory([self.config.artifacts_root]) #artifacts directory will get created

    #now initialize the class variable value to it-->for which we are creating method
    def get_callback_config(self) ->CallBackModelConfig:
        self.config = self.config.prepare_callback_model #rtn value as config dictatonary

        self.chkpt_directory = os.path.dirname(self.config.model_checkpoint_path)

        #creating directory artifacts/callback_model 
        Create_Directory([self.config.root_dir_path,self.chkpt_directory])

        #now initializing value to CallBackModelConfig class variable and taking rtn as function creating an object first
        call_back_config = CallBackModelConfig(
            root_dir_path=self.config.root_dir_path,
            tensorboard_log_dir_path=self.config.tensorboard_log_dir_path,
            model_checkpoint_path=self.config.model_checkpoint_path
        )

        return call_back_config

In [5]:
from datetime import datetime

## step5) update the component file

In [6]:
#in component file we gonna initialize object to CallBackModelConfig class 
#and accessing all the parameter and value init

class CallBackModel():
    def __init__(self,config:CallBackModelConfig):
        self.config = config


    #create method to get tensorboard log directory
    @property
    def _get_tensorboard_log_dir(self):
        timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

        tensorboard_log_dir = os.path.join(self.config.tensorboard_log_dir_path,f"TB_{timestamp}.log")

        # Ensure the directory exists
        os.makedirs(os.path.dirname(tensorboard_log_dir), exist_ok=True)

        tb_directory = tensorflow.keras.callbacks.TensorBoard(
            log_dir=tensorboard_log_dir
        )
        return tb_directory
    
    #method to create model_checkpoint to save best model file after training done
    @property
    def _get_checkpont_model(self):
        checkpoint_model = tensorflow.keras.callbacks.ModelCheckpoint(
            filepath=self.config.model_checkpoint_path,
            verbose=1,
            save_best_only=True
        )
        return checkpoint_model



    def callback(self):
        return[
        self._get_tensorboard_log_dir,
        self._get_checkpont_model
        ]

In [7]:
os.chdir('../')
%pwd

'd:\\DL MODEL\\ImageClassificationModel'

## step6) updating the pipeline file

In [8]:
try:
    #creating an object of ConfigurationManager class
    cm = ConfigurationManager()

    #calling the method of class
    call_back_config = cm.get_callback_config()

    #creating an object of callbacks class
    cb = CallBackModel(config=call_back_config)

    #calling teh mthod of callbacks class
    lst_obj = cb.callback()

    logger.info(lst_obj)

except Exception as e:
    raise CustomException(e,sys)


[2024-09-10 17:50:01,756]-INFO-66-Reading the YAML file config\config.yaml
[2024-09-10 17:50:01,765]-INFO-69-YAML file read successfully: config\config.yaml
[2024-09-10 17:50:01,767]-INFO-66-Reading the YAML file params.yaml
[2024-09-10 17:50:01,772]-INFO-69-YAML file read successfully: params.yaml
[2024-09-10 17:50:01,775]-INFO-80-Creating Directory
[2024-09-10 17:50:01,776]-INFO-80-Creating Directory
[2024-09-10 17:50:01,892]-INFO-14-[<keras.src.callbacks.TensorBoard object at 0x000001EAD01701C0>, <keras.src.callbacks.ModelCheckpoint object at 0x000001EAD01517C0>]
